In [1]:
import xarray as xr
import dask.array as da
import numpy as np

import inspire_agrivolt

In [ ]:
model_outputs = xr.open_zarr("/projects/inspire/PySAM-MAPS/v0.5/colorado/08.zarr")

In [ ]:
# downselect toy problem subset
scenario_dataset = model_outputs.isel(gid=slice(0,10))
# set values to check binning size
scenario_dataset.pitch[0]=3.8
scenario_dataset.pitch[1]=3.81
scenario_dataset.pitch[5]=10
scenario_dataset.pitch[8]=11.5
scenario_dataset.pitch[9]=12

scenario_dataset

In [ ]:
scenario_dataset.data_vars

In [ ]:
def iter_beds(scenario_dataset: xr.Dataset) -> xr.DataArray:
    
    bins = np.array([3.8, 3.8492,  4.9491, 6.9355, 11.5465, 12.0001, np.inf ])
    distance_index = np.arange(0, 10, dtype=int)
    
    slices = np.array(
            [
                [0, 5, 5, 7, 7, 8, 8, 10], # 3.8, 3.8492
                [0, 4, 4, 6, 6, 8, 8, 10], # 3.8492,  4.9491
                [0, 3, 3, 5, 5, 8, 8, 10], # 4.9491, 6.9355
                [0, 2, 2, 5, 5, 7, 7, 10], # 6.9355, 11.5465
                [0, 1, 1, 4, 4, 7, 7, 10], # 11.5465, 12.0
            ],
        dtype=int
    )
    
    digitized = np.digitize(scenario_dataset.pitch, bins)
    
    # if anything in the range
    if np.any(digitized == 6):
        raise ValueError("Invalid pitch value above 12 provided.")
    
    slice_index = digitized - 1
    selected_slices = slices[slice_index]
    
    res = []
    for i, gid in enumerate(scenario_dataset.gid):
        # calculate mask
        under_start= selected_slices[i,0]
        under_end  = selected_slices[i,1]
        beda_start = selected_slices[i,2]
        beda_end   = selected_slices[i,3]
        bedb_start = selected_slices[i,4]
        bedb_end   = selected_slices[i,5]
        bedc_start = selected_slices[i,6]
        bedc_end   = selected_slices[i,7]
        
        mask_under = (distance_index >= under_start) & (distance_index < under_end)
        mask_beda = (distance_index >= beda_start) & (distance_index < beda_end) 
        mask_bedb = (distance_index >= bedb_start) & (distance_index < bedb_end)
        mask_bedc = (distance_index >= bedc_start) & (distance_index < bedc_end)
        
        single_loc_beds_ds = xr.merge(
            [
            scenario_dataset.ground_irradiance.isel(gid=i, distance=mask_under).mean(dim="distance").rename("under_pannel"),
            scenario_dataset.ground_irradiance.isel(gid=i, distance=mask_beda).mean(dim="distance").rename("beda"),
            scenario_dataset.ground_irradiance.isel(gid=i, distance=mask_bedb).mean(dim="distance").rename("bedb"),
            scenario_dataset.ground_irradiance.isel(gid=i, distance=mask_bedc).mean(dim="distance").rename("bedc"),
            ]
        )
        
        res.append(single_loc_beds_ds)
        
    return xr.concat(res, dim="gid")

In [ ]:
chunks = scenario_dataset.chunks

chunks_gid = chunks["gid"][0]
chunks_time = chunks["time"][0]

dims = scenario_dataset.dims

size_gid = chunks["gid"][0]
size_time = chunks["time"][0]

beds_dims = ("gid", "time")

# change to lambda function?
def beds_array():
    return da.empty(dtype=float, shape=(size_gid, size_time), chunks=(chunks_gid, chunks_time))

map_template = xr.Dataset(
    data_vars = {
        'under_pannel': (beds_dims, beds_array()),
        'beda': (beds_dims, beds_array()),
        'bedb': (beds_dims, beds_array()),
        'bedc': (beds_dims, beds_array())
    },
    coords={
        "gid":scenario_dataset.gid,
        "time":scenario_dataset.time,
    }
)

In [ ]:
beds_ds = scenario_dataset.map_blocks(iter_beds, template=map_template)

In [ ]:
beds_ds.compute()

In [ ]:
beds_ds.isel(gid=0).beda.plot()
beds_ds.isel(gid=0).bedb.plot()
beds_ds.isel(gid=0).bedc.plot()

In [ ]:
(scenario_dataset["annual_energy"]).compute()

In [ ]:
(scenario_dataset["annual_energy"] / 80.466).compute()

In [ ]:
scenario_dataset